In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().magic(u'matplotlib inline')
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')

In [103]:
import numpy as np

In [2]:
data = pd.read_csv('result資料.csv')

A:\Anaconda\envs\gaf\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,24,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data['ymd'] = pd.to_datetime(data['ymd'])

In [6]:
data['first_ymd'] = data.groupby('billing_email')['ymd'].transform('min')

In [124]:
before_2 = data.query('ymd <= "2016-02-28"')
before_3 = data.query('ymd <= "2016-03-31"')
before_4 = data.query('ymd <= "2016-04-30"')
before_5 = data.query('ymd <= "2016-05-31"')
before_6 = data.query('ymd <= "2016-06-30"')

In [71]:
def GetRFM(data):
    nowday = pd.to_datetime(data['ymd'].max())

    data['lastdate'] = pd.to_datetime(data.groupby('billing_email')['ymd'].transform('max'))

    data['R'] = nowday - data['lastdate']
    data['R'] = data['R'].astype('str').str.split('d').str[0].astype(int)
    data['M'] = data.groupby('billing_email')['order_total'].transform('sum')
 
    dailydate = data.drop_duplicates(subset=['billing_email', 'ymd'])
    dailydate['F'] = dailydate.groupby('billing_email')['ymd'].transform('count')
    
    return dailydate

In [30]:
def RepPeriod(data):
    repurchase = data.query('F > 1')
    repurchase = repurchase.sort_values(by=['billing_email', 'ymd'])
    repurchase['lastymd'] = repurchase.groupby('billing_email')['ymd'].shift().fillna(repurchase['ymd'])
    repurchase['repurchase_period'] = pd.to_datetime(repurchase['ymd']) - pd.to_datetime(repurchase['lastymd'])
    repurchase['repurchase_period'] = repurchase['repurchase_period'].astype('str').str.split('d').str[0].astype(int)
    
    RepPeriod = repurchase.query('repurchase_period > 0')
    return RepPeriod['repurchase_period'].mean() * 3

In [155]:
def GetNAPL(data):
    data['Frequent'] = data['F'].apply(FreqType)
    data['napl'] = data.apply(lambda x: Grouped(x['R'], x['F']), axis=1)
    money = data.groupby(['napl'])['M'].sum()
    people = data.groupby(['napl'])['billing_email'].count()
    return (data, money, people)

In [34]:
def FreqType(x):
    if x == 1:
        return '1'
    if x == 2:
        return '2'
    if x > 2:
        return '3 up'

In [36]:
def Grouped(x, y):
    if y > 1:
        if x > 169:
            return 'potential'
        else:
            return 'active'
    else:
        if x > 169:
            return 'lost'
        else:
            return 'new'

In [126]:
data2 = GetRFM(before_2)
data3 = GetRFM(before_3)
data4 = GetRFM(before_4)
data5 = GetRFM(before_5)
data6 = GetRFM(before_6)

In [33]:
rep_mean = RepPeriod(data3)
print(int(rep_mean))

169


In [127]:
data2_unique = data2.drop_duplicates(subset='billing_email')
data3_unique = data3.drop_duplicates(subset='billing_email')
data4_unique = data4.drop_duplicates(subset='billing_email')
data5_unique = data5.drop_duplicates(subset='billing_email')
data6_unique = data6.drop_duplicates(subset='billing_email')

In [157]:
data2, money2, people2 = GetNAPL(data2_unique)
data3, money3, people3 = GetNAPL(data3_unique)
data4, money4, people4 = GetNAPL(data4_unique)
data5, money5, people5 = GetNAPL(data5_unique)
data6, money6, people6 = GetNAPL(data6_unique)

In [159]:
ymd = pd.Series(np.zeros(money2.shape[0]), name='ymd')
df = money2.to_frame().join(ymd).join(people2).reset_index()
df['ymd'] = '2016-02'
df['M_percentage'] = df['M'] / df['M'].sum() * 100
df['N_percentage'] = df['billing_email'] / df['billing_email'].sum() * 100
df

,napl,M,ymd,billing_email,M_percentage,N_percentage
0,active,9710486,2016-02,1150,49.521052,22.209347
1,lost,1417314,2016-02,666,7.227947,12.862109
2,new,7568428,2016-02,3219,38.597091,62.166860
3,potential,912576,2016-02,143,4.653910,2.761684


In [167]:
tempdf = money6.to_frame().join(ymd).join(people6).reset_index()
tempdf['ymd'] = '2016-06'
tempdf['M_percentage'] = tempdf['M'] / tempdf['M'].sum() * 100
tempdf['N_percentage'] = tempdf['billing_email'] / tempdf['billing_email'].sum() * 100
tempdf

,napl,M,ymd,billing_email,M_percentage,N_percentage
0,active,23907603,2016-06,2336,54.304388,23.264615
1,lost,4718568,2016-06,2101,10.717885,20.924211
2,new,12334822,2016-06,5173,28.017655,51.518773
3,potential,3064184,2016-06,431,6.960072,4.292401


In [168]:
df = pd.concat([df, tempdf], axis=0)
df

,napl,M,ymd,billing_email,M_percentage,N_percentage
0,active,9710486,2016-02,1150,49.521052,22.209347
1,lost,1417314,2016-02,666,7.227947,12.862109
2,new,7568428,2016-02,3219,38.597091,62.166860
3,potential,912576,2016-02,143,4.653910,2.761684
0,active,13070547,2016-03,1518,49.435377,22.415830
1,lost,2168870,2016-03,816,8.203092,12.049616
2,new,10030950,2016-03,4265,37.939024,62.979917
3,potential,1169296,2016-03,173,4.422507,2.554637
0,active,17079146,2016-04,1839,53.186615,23.399924
1,lost,2332333,2016-04,914,7.263179,11.629978


In [169]:
df.to_csv('trytry.csv')